--- 
Project for the course in Microeconometrics | Summer 2021, M.Sc. Economics, Bonn University | [Philipp Schreiber](https://github.com/pcschreiber1)

# Replication of Henderson, Storeygard, Deichmann (2017) :
## Has climate change driven urbanization in Africa? <a class="tocSkip">   
    
    
---

<div class="alert alert-block alert-info">
    <h5><b>Downloading and viewing this notebook:</b></h5> <p> The original paper, as well as the data and code provided by the authors can be accessed <a href="https://doi.org/10.1016/j.jdeveco.2016.09.001">here</a> </p></div>

## Content:
Extension: 
 - Spatial SUTVA/ <span style="color:orange">**NOTE**:</span> How large are the districts (important for whetehr SUTVA is reasonable or not)
 - Spatially explicit counterfactual framework
     - using exploration of SUTVA and spatial patters
         - spatial autocorrelation: Moran Plot and Moran's I
     - implementing (lagged dependent, lagged 

In [1]:
%matplotlib inline

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker
import statsmodels.formula.api as smf
import seaborn as sns

#For spatial analysis
import geopandas as gpd
import shapely.geometry as geom
import libpysal as lp #For spatial weights

from pysal.viz import splot #exploratory analysis
#from splot.esda import plot_moran #exploratory analysis
from pysal.explore import esda #exploratory analysis
from pysal.model import spreg #For spatial regression

pd.options.display.float_format = "{:,.2f}".format

In [ ]:
from auxiliary.data_import import *
from auxiliary.plots import *

In [1]:
#map_countries()
regiondata = get_shapefile()

NameError: name 'get_shapefile' is not defined

In [ ]:
# The explanatory variables of interest
a = ["ADsm0_1moistu",
     "ADsm0_2moistu",
     "ADsm0_3moistu",
     "ADsm0_4moistu",
     "extent_agE_ADsm0_2moistu",
     "ADurbfrac",
     "extent_agE",
     "extent_agH",
     "primate"]

---
# 1. Introduction 
---

In this paper Henderson et al. investigate the impact of increasing aridity on urbanization in Sub-Saharan Africa. In particular, the authors address the questions of i) *whether adverse changes in climate induce a push from rural to urban areas* and ii) *how this within-country migration affects incomes in cities/ the nature of the development effect of such an urbanization push on cities*.  They attribute previous studies' lack of significant findings to aggregation due to using national level data and, therefore, use variation at the district and city level. In addition, they construct a new data set of actual census data to overcome the reliance on interpolations, including the use of **night time lights (NTLs)**.

<img src="material/map_countries.PNG" alt="Error: map_countries.png" title="countries in the data set" width="600"/>

The authors outline a simple model of rural-urban migration to frame their analysis of the impact of rising aridity. The model suggests ...
<span style="color:red">***Shorter*** </span> <span style="color:blue">The authors model districs as small open economies which are differentiated by the presence of manufacturing facilities for potentially tradeable goods. In this context, a decline in moisture should only induce local migration in industrialized districts, since in exclusievly rural districts cities are only provide agriculture with services that not traded across districts, the local agricultural sector is not competing for labor.</span> This study employs **fixed effects (FE) regression** to account for all relevant influences. Finally, the authors support this by looking at evidence from India. The authors find strong evidence for the local migration, nut only in districts likely to produce tradable manufacturing goods. Similary, total income of local cities also only increases in districts liekly to produce tradable manufacturing goods. Understanding the patterns and effects of climate change on urbanization  is important both for providing assistance and to guide proactive policy. Africa projected to be especially affected by climate change ... Sucessful spatial and structural adaptation to climate change will critically depend on promoting cities' capacities.



<span style="color:blue">In this notebook, I replicate the results presented in the paper by Lindo et al. (2010). Furthermore, I discuss in detail the identification strategy used by the authors and evaluate the results using multiple robustness checks. My analysis offers general support for the findings of Lindo et al. (2010) and points out some factors which may enable a deeper understanding of the causal relationship explored in the paper. </span> In particular, I show that the SUTVA assumption is violated through spatial interdependence and **introduce a spatially explicity counterfactual model as an alternative**.

<span style="color:blue"> This notebook is structured as follows. In the next section, I present the performance standard model that lays down the theoretical framework for the paper (Section 2). In Section 3, I analyze the identification strategy that Lindo et al. (2010) use to unravel the causal effects of academic probation on student outcomes and Section 4 briefly discusses the empirical strategy the authors use for estimation. Section 5 and Section 6 constitute the core of this notebook. Section 5 shows my replication of the results in the paper and discussion thereof. In Section 6 I conduct various robustness checks and discuss some limitations of the paper. Section 7 offers some concluding remarks. </span>

